# 表示回路２ユーザプログラム
- これまでの表示回路２の機能とサンプルプログラムを参照して、自由にプログラムを作成してください

### [STEP-1] 定数定義
- 表示解像度パラメータ（VGA or XGA）
- レジスタアドレス設定

In [1]:
# 表示解像度パラメータ
PIXEL_VGA_WIDTH  = 640
PIXEL_VGA_HEIGHT = 480

PIXEL_XGA_WIDTH  = 1024
PIXEL_XGA_HEIGHT = 768

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [2]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 表示回路１のコンフィギュレーション

In [3]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] パターン表示回路の解像度設定
- RESOLの値を変更してパターン表示回路の解像度を設定
    - 0 : VGA設定
    - 1 : XGA設定
    - 2 : SXGA設定
- 表示回路２では、RESOLの値は0,1のみ対応

In [29]:
RESOL = 1
pl.axi_gpio_0.write(RA_GPIO_DATA,(RESOL&1))
pl.axi_gpio_0.read(RA_GPIO_DATA)

1

### [STEP-5] DisplayPort初期化
- パターン表示回路の解像度に設定を合わせる
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)
    - XGA設定  : dp.configure(VideoMode(1024,768,24),PIXEL_RGB)
    - SXGA設定 : dp.configure(VideoMode(1280,1024,24),PIXEL_RGB)

In [30]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

if (RESOL & 1) == 1 :
    PIXEL_WIDTH  = PIXEL_XGA_WIDTH
    PIXEL_HEIGHT = PIXEL_XGA_HEIGHT
    print("XGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
else :
    PIXEL_WIDTH  = PIXEL_VGA_WIDTH
    PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
    print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

XGA setting @ DisplayPort 1024 768


### [STEP-6] ユーザプログラム

### [STEP-99] 終了処理
- 表示OFF @ DISPCTRL
- PYNQの初期画面に戻す

In [35]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)
lvdp.close()